## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3): (num, length, width, channel)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 2s 0us/step
(50000, 32, 32, 3)


In [0]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same', input_shape=(32,32,3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 119s 2ms/step - loss: 1.4296 - accuracy: 0.5171
Epoch 2/100
50000/50000 [==============================] - 119s 2ms/step - loss: 0.9851 - accuracy: 0.6566
Epoch 3/100
50000/50000 [==============================] - 119s 2ms/step - loss: 0.8295 - accuracy: 0.7080
Epoch 4/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.7076 - accuracy: 0.7494
Epoch 5/100
50000/50000 [==============================] - 121s 2ms/step - loss: 0.5958 - accuracy: 0.7915
Epoch 6/100
50000/50000 [==============================] - 117s 2ms/step - loss: 0.5018 - accuracy: 0.8226
Epoch 7/100
50000/50000 [==============================] - 119s 2ms/step - loss: 0.4174 - accuracy: 0.8551
Epoch 8/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.3483 - accuracy: 0.8796
Epoch 9/100
50000/50000 [==============================] - 118s 2ms/step - loss: 0.2852 - accuracy: 0.8998
Epoch 10/100
50000/50000 [===========

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [25]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.0954865e-02, 3.0239607e-05, 8.0091096e-02, 8.3005178e-01,
        6.2804259e-02, 3.7788088e-03, 2.0480235e-03, 1.9478379e-04,
        9.9539589e-03, 9.2135968e-05]], dtype=float32)

In [46]:
n_tested=100
is_selected=np.random.randint(10000, size=n_tested)
pred=classifier.predict(x_test[is_selected,:,:,:])
n_correct=0
for i in range(0,pred.shape[0]):
  print('prediction:',np.argmax(pred[i]))
  print('y:',np.argmax(y_test[is_selected,][i]))
  if np.argmax(pred[i]) == np.argmax(y_test[is_selected,][i]):
    n_correct = n_correct+1

print('accuracy:',n_correct/pred.shape[0])

prediction: 2
y: 2
prediction: 4
y: 3
prediction: 7
y: 7
prediction: 9
y: 9
prediction: 9
y: 9
prediction: 3
y: 5
prediction: 7
y: 4
prediction: 7
y: 4
prediction: 1
y: 1
prediction: 8
y: 8
prediction: 6
y: 3
prediction: 6
y: 6
prediction: 0
y: 0
prediction: 0
y: 0
prediction: 2
y: 2
prediction: 7
y: 4
prediction: 1
y: 1
prediction: 7
y: 5
prediction: 9
y: 9
prediction: 2
y: 0
prediction: 0
y: 8
prediction: 5
y: 5
prediction: 6
y: 6
prediction: 8
y: 8
prediction: 7
y: 5
prediction: 6
y: 6
prediction: 6
y: 6
prediction: 7
y: 7
prediction: 1
y: 1
prediction: 6
y: 6
prediction: 7
y: 7
prediction: 8
y: 8
prediction: 9
y: 7
prediction: 8
y: 8
prediction: 7
y: 7
prediction: 1
y: 8
prediction: 7
y: 5
prediction: 9
y: 0
prediction: 8
y: 0
prediction: 7
y: 7
prediction: 3
y: 3
prediction: 3
y: 4
prediction: 0
y: 0
prediction: 4
y: 4
prediction: 1
y: 9
prediction: 4
y: 4
prediction: 7
y: 7
prediction: 5
y: 5
prediction: 0
y: 0
prediction: 5
y: 5
prediction: 7
y: 4
prediction: 0
y: 7
prediction: 